In [ ]:
# 环境配置
# 配置python3.9环境
# %%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)
# 注：以上代码执行完成后，需点击左上角或右上角将kernel更换为python-3.9.0



In [ ]:
# 安装mindspore2.2.14，安装指南详见：MindSpore安装
# 安装MindNLP及相关依赖，MindNLP官方仓详见：MindNLP
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.14/MindSpore/unified/x86_64/mindspore-2.2.14-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install tokenizers==0.15.2 -i https://pypi.tuna.tsinghua.edu.cn/simple
!wget https://repo.mindspore.cn/mindspore-lab/mindnlp/newest/any/mindnlp-0.3.2-py3-none-any.whl
!pip install mindnlp-0.3.2-py3-none-any.whl

In [ ]:
from mindnlp.transformers import AutoModelForSeq2SeqLM
from mindnlp.peft import LoraConfig, TaskType, get_peft_model

# Load the pre-trained model
model_name_or_path = "bigscience/mt0-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

# Get the model with a PEFT adapter
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)

# Print the trainable parameters of the model
model.print_trainable_parameters()

`LoraConfig` specifies how the PEFT adapters should be configured:

* `task_type`: Defines the type of task, which in this case is TaskType.SEQ_2_SEQ_LM for sequence-to-sequence language modeling.
* `inference_mode`: A boolean that should be set to False when training to enable the training-specific features of the adapters.
* `r`: Represents the rank of the low-rank matrices that are part of the adapter. A lower rank means less complexity and fewer parameters to train.
* `lora_alpha`: LoRA alpha is the scaling factor for the weight matrices. A higher alpha value assigns more weight to the LoRA activations.
* `lora_dropout`: Sets the dropout rate within the adapter layers to prevent overfitting.

## Prepare the Dataset

To fine-tune the model, let's use the [financial_phrasebank](https://huggingface.co/datasets/takala/financial_phrasebank) dataset. The financial_phrasebank dataset is specifically designed for sentiment analysis tasks within the financial sector. It contains sentences extracted from financial news articles, which are categorized based on the sentiment expressed — negative, neutral or positive.

Although the dataset is designed for sentiment classification task, we use it here for a sequence-to-sequence task for simplicity.

### Load the dataset
Load the dataset with `load_dataset` from MindNLP.

The data is then shuffled and split, allocating 90% for training and 10% for validation.

In [7]:
from data_process import LotteryDataLoader

my_dataset = LotteryDataLoader()

import mindspore.dataset as ds
dataset = ds.GeneratorDataset(my_dataset, ["enquire", "answer"], shuffle=True)

train_dataset, validation_dataset = dataset.shuffle(64).split([0.9, 0.1])

[WARNING] ME(25078:139755628070080,MainProcess):2024-07-11-15:30:14.394.536 [mindspore/dataset/engine/datasets.py:1203] Dataset is shuffled before split.


### Tokenization
We then tokenize the text with the tokenizer associated with the mT0 model. First, load the tokenizer:

In [8]:
from mindnlp.transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

/home/ros/miniconda3/envs/huawei/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.542 seconds.
Prefix dict has been built successfully.


NameError: name 'model_name_or_path' is not defined

Next, modify the `BaseMapFunction` from MindNLP to wrap up the tokenization steps.

Note that both the `sentence` and the `text_label` columns needs to be tokenized.

In addition, to avoid unexpected behavior due to multiple threads attempting to tokenize data at the same time, we use `Lock` from the `threading` module to ensure only one thread can perform the tokenization at a time.

In [ ]:

import numpy as np
from mindnlp.dataset import BaseMapFunction
from threading import Lock
lock = Lock()

max_length = 128
class MapFunc(BaseMapFunction):
    def __call__(self, sentence, label, text_label):
        lock.acquire()
        model_inputs = tokenizer(sentence, max_length=max_length, padding="max_length", truncation=True)
        labels = tokenizer(text_label, max_length=max_length, padding="max_length", truncation=True)
        lock.release()
        labels = labels['input_ids']
        labels = np.where(np.equal(labels, tokenizer.pad_token_id), -100, labels)
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels

Next, we apply the map function, shuffle the dataset if necessary and batch the dataset:

In [ ]:

def get_dataset(dataset, tokenizer, batch_size=None, shuffle=True):
    input_colums=['enquire']
    output_columns=['answer']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    if shuffle:
        dataset = dataset.shuffle(64)
    if batch_size:
        dataset = dataset.batch(batch_size)
    return dataset

batch_size = 8
train_dataset = get_dataset(train_dataset, tokenizer, batch_size=batch_size)
eval_dataset = get_dataset(validation_dataset, tokenizer, batch_size=batch_size, shuffle=False)

## Train the model

Now we have the model and datasets ready, let's prepare for the training.

### Optimizer and learning rate scheduler

We set up the optimizer for updating the model parameters, alongside a learning rate scheduler that manages the learning rate throughout the training process.

In [ ]:
from mindnlp.modules.optimization import get_linear_schedule_with_warmup
import mindspore.experimental.optim as optim

# Setting up optimizer and learning rate scheduler
optimizer = optim.AdamW(model.trainable_params(), lr=1e-3)

num_epochs = 3 # Number of iterations over the entire training dataset
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=(len(train_dataset) * num_epochs))

### Training step

Next, define the function that controls each step of training.

Define `forward_fn` which executes the model's forward pass to compute the loss.

Then pass `forward_fn` to `mindspore.value_and_grad` to create `grad_fn` that computes both the loss and gradients needed for parameter updates.

Define `train_step` that updates the model's parameters according to the computed gradients, which will be called in each step of training.

In [ ]:
import mindspore
from mindspore import ops

# Forward function to compute the loss
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

# Gradient function to compute gradients for optimization
grad_fn = mindspore.value_and_grad(forward_fn, None, model.trainable_params())

# Define the training step function
def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer(grads)  # Apply gradients to optimizer for updating model parameters
    return loss

### Training loop

Now everything is ready, let's implement the training and evaluation loop and excute the training process.

This process optimizes the model's parameters through multiple iterations over the dataset, i.e. multiple epochs, and assesses its performance on the evaluation dataset.

In [ ]:
from tqdm import tqdm

# Training loop across epochs
for epoch in range(num_epochs):
    model.set_train(True)
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    # Iterate over each entry in the training dataset
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        loss = train_step(**batch)
        total_loss += loss.float()  # Accumulate loss for monitoring
        lr_scheduler.step()  # Update learning rate based on scheduler

    model.set_train(False)
    eval_loss = 0
    eval_preds = []
    eval_total_size = eval_dataset.get_dataset_size()
    # Iterate over each entry in the evaluation dataset
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with mindspore._no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.float()
        eval_preds.extend(
            tokenizer.batch_decode(ops.argmax(outputs.logits, -1).asnumpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss) # Perplexity
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss) # Perplexity
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

Let's break down the training loop implementation and understand the key components:

* Model Training Mode

    Before the training starts, the model is set to training mode by `model.set_train(True)`. Before the evaluation, the training-specific behaviour of the model is disabled by `model.set_train(False)`.

* Loss and Perplexity

    `total_loss = 0` initializes and `total_loss += loss.float()` accumulates the total loss for each batch within an epoch. This accumulation is crucial for monitoring the model’s performance.

    The average loss and the perplexity (PPL), which is a common metric for language models, are reported in the printed message.

* Learning Rate Scheduler

    `lr_scheduler.step()` adjusts the learning rate after processing each batch, according to the predefined schedule. This is vital for effective learning, helping to converge faster or escape local minima.

* Evaluation Loop

    During evaluation, in addition to `model.set_train(False)`, `mindspore._no_grad()` ensures that gradients are not computed during the evaluation phase, which conserves memory and computations.
    The `tokenizer.batch_decode()` function converts the output logits from the model back into readable text. This is useful for inspecting what the model predicts and for further qualitative analysis.

## After training
Now that we have finished the training, we can assess its performance and save the trained model for future use.

### Accuracy compuation and check the predicited results

Let's comupute the accuracy of the predictions made on the validation dataset. Accuracy is a direct measure of how often the model's predictions match the actual labels, providing a straightforward metric to reflect the model's effectiveness.

In [ ]:
# Initialize counters for correct predictions and total predictions
correct = 0
total = 0

# List to store actual labels for comparison
ground_truth = []

# Compare each predicted label with the true label
for pred, data in zip(eval_preds, validation_dataset.create_dict_iterator(output_numpy=True)):
    true = str(data['text_label'])
    ground_truth.append(true)
    if pred.strip() == true.strip():
        correct += 1
    total += 1

# Calculate the percentage of correct predictions
accuracy = correct / total * 100

# Output the accuracy and sample predictions for review
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{ground_truth[:10]=}")

### Saving the model
If you are satisfied with the result, you can save the model as follows:

In [ ]:
# Save the model
peft_model_id = f"../../output/{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

## Use the model for inference

Now let's load the saved model and demonstrate how to use it for making predictions on new data.

To load the model that has been trained with PEFT, we first load the base model with `AutoModelForSeq2SeqLM.from_pretrained`. On top of it, we add the trained PEFT adapter to the model with `PeftModel.from_pretrained`:

In [ ]:
from mindnlp.transformers import AutoModelForSeq2SeqLM
from mindnlp.peft import PeftModel, PeftConfig

peft_model_id = f"../../output/{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

# Load the model configuration
config = PeftConfig.from_pretrained(peft_model_id)

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)

# Load the pretrained adapter
model = PeftModel.from_pretrained(model, peft_model_id)

Next, retrieve an entry from the validation dataset, or alternatively create an entry on your own.

We tokenize the `'sentence'` in this entry and use it as inputs into the model. Excute it and be curious about what the model will predict.

In [ ]:
# Retrieve an entry from the validation dataset.
# example = next(validation_dataset.create_dict_iterator(output_numpy=True)) # Get an example entry from the validation dataset
# print(example['sentence'])
# print(example['text_label'])

# Alternatively, create your own text
example = {'sentence': 'Nvidia Tops $3 Trillion in Market Value, Leapfrogging Apple.'}

inputs = tokenizer(example['sentence'], return_tensors="ms") # Get the tokenized text label
print(inputs)

model.set_train(False)
with mindspore._no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10) # Predict the text label using the trained model
    print(outputs)
    print(tokenizer.batch_decode(outputs.asnumpy(), skip_special_tokens=True)) # Print decoded text label from the prediction

In [ ]:
import os
from typing import Dict, Union, Optional
from typing import List


import mdtex2html

#将文本中的字符转为网页上可以支持的字符，避免被误认为是HTML标签
def parse_text(text):
    """copy from https://github.com/GaiZhenbiao/ChuanhuChatGPT/"""
    lines = text.split("\n")
    lines = [line for line in lines if line != ""]
    count = 0
    for i, line in enumerate(lines):
        if "```" in line:
            count += 1
            items = line.split('`')
            if count % 2 == 1:
                lines[i] = f'<pre><code class="language-{items[-1]}">'
            else:
                lines[i] = f'<br></code></pre>'
        else:
            if i > 0:
                if count % 2 == 1:
                    line = line.replace("`", "\`")
                    line = line.replace("<", "&lt;")
                    line = line.replace(">", "&gt;")
                    line = line.replace(" ", "&nbsp;")
                    line = line.replace("*", "&ast;")
                    line = line.replace("_", "&lowbar;")
                    line = line.replace("-", "&#45;")
                    line = line.replace(".", "&#46;")
                    line = line.replace("!", "&#33;")
                    line = line.replace("(", "&#40;")
                    line = line.replace(")", "&#41;")
                    line = line.replace("$", "&#36;")
                lines[i] = "<br>"+line
    text = "".join(lines)
    return text

In [ ]:
# 编写Gradio调用函数

#采用流聊天方式（stream_chat）调用模型，使得生成答案有逐字生成的效果
def predict(input, chatbot, max_length, top_p, temperature, history):
    chatbot.append((parse_text(input), ""))
    for response, history in model.stream_chat(tokenizer, input, history, max_length=max_length, top_p=top_p,
                                               temperature=temperature):
        chatbot[-1] = (parse_text(input), parse_text(response))       

        yield chatbot, history

#去除输入框的内容
def reset_user_input():
    return gr.update(value='')

#清除状态
def reset_state():
    return [], [], None

In [ ]:
#运行Gradio界面，运行成功后点击“Running on public URL”后的网页链接即可体验
import gradio as gr

with gr.Blocks() as demo:
    gr.HTML("""<h1 align="center">MindNLP ChatGLM-6B StreamChat</h1>""")

    chatbot = gr.Chatbot()
    with gr.Row():
        with gr.Column(scale=4):
            with gr.Column(scale=12):
                user_input = gr.Textbox(show_label=False, placeholder="Input...", lines=3, container=False)
            with gr.Column(min_width=32, scale=1):
                with gr.Row():
                    submitBtn = gr.Button("一键开聊", variant="primary")
                    emptyBtn = gr.Button("清除历史")
            with gr.Column(scale=1):
                max_length = gr.Slider(0, 4096, value=2048, step=1.0, label="Maximum length", interactive=True)
                top_p = gr.Slider(0, 1, value=0.7, step=0.01, label="Top P", interactive=True)
                temperature = gr.Slider(0, 1, value=0.95, step=0.01, label="Temperature", interactive=True)               

    history = gr.State([])

    submitBtn.click(predict, [user_input, chatbot, max_length, top_p, temperature, history], [chatbot, history],
                    show_progress=True)
    submitBtn.click(reset_user_input, [], [user_input])

    emptyBtn.click(reset_state, outputs=[chatbot, history], show_progress=True)

demo.queue().launch(share=True)